# Introduction

作者：Cholian

时间：20210823

所属解释权：蓝楹会研究部（量化出品）

logs:
    
    --**20211121 Cholian

tushare changed the industry code from "SW" to "SW2021", done

# Package

In [1]:
import tushare as ts
import pandas as pd
# import matplotlib.pyplot as plt
# import datetime
import numpy as np
import time
import datetime

如果报错“no packge called tushare”，请咨询技术部，或者联系cholian，WeChat: cholian0518

这里的API接口请联系技术部询问tushare数据库API

In [2]:
# 这里填写一长串数据库api密钥，e.g: pro = ts.pro_api('asdasdjlasjdlkasjd123123213')
pro = ts.pro_api('cb8a90fa4e956282996a0eb8674ed01af79dd27145f039febd4e69d5')

## 参数选取

##### --------------------------------- <font color='red'>需要手动更新的地方 start </font>------------------------------------------------------

### 日期参数 （每周更新）

In [3]:
# 日期参数
# 提取上周五股票的原因是因为，周一要计算净流入的话，就要和上周五进行对比，因此，需要额外的把上周五的也提取出来
start_date ="20211112" # 上周五的日期
end_date = "20211119" # 最新周五的日期

### 行业股票参数（可选）

In [4]:
# 选取前几名行业 （默认为5）
top_industry = 5

# 选取前几名的股票
top_share = 5

##### ---------------------------------<font color='blue'>需要手动更新的地方 end </font> ------------------------------------------------------

## 准备函数

In [5]:
day = (pd.to_datetime(end_date)- pd.to_datetime(start_date)).days

In [6]:
def sw_indurstry( level = "L1",pro = None):
    #获取申万一级行业列表
    df_first = pro.index_classify(level=level, src='SW2021')
    df_first_detail = pd.DataFrame()
    
    for i in df_first['index_code']:

        a = pro.index_member(index_code=i)
        
        df_first_detail = pd.concat([a,df_first_detail])
        
    return df_first_detail

def trade_datetime(end_date='20210714',period = 7):
    
    end_date = pd.to_datetime(end_date)
    dates = end_date - pd.to_timedelta(np.arange(period), unit = 'D')
    dates = dates.strftime('%Y%m%d')

    return dates


def pivot_data(data,col_pivot,dates_pivot,value):
    dates = pd.unique(data[dates_pivot])
    result = pd.DataFrame(columns=pd.unique(data[col_pivot]))
    result['date'] = ''
    
    for i in dates:

        daily = data[data[dates_pivot] == i]
        data_pivot = daily.groupby(col_pivot)[value].sum()
        
        a = pd.DataFrame(data_pivot).T
        a['date'] = i
        result = pd.concat([result,a])
    return result

def find_top(n,data):
    top_list = pd.DataFrame()
    for i,day in enumerate(data.index):
        top_data = data.iloc[i].sort_values(ascending = False).head(n).index
        top_list[day] = top_data
        
    return top_list

In [7]:
%%time
# 申万行业数据

df_first_detail = sw_indurstry( level = "L1",pro = pro)
df_second_detail = sw_indurstry( level = "L2",pro = pro)

CPU times: user 1.64 s, sys: 274 ms, total: 1.91 s
Wall time: 2min 2s


In [8]:
%%time
data_all = pd.DataFrame()


dates = trade_datetime(end_date=end_date,period = day+1)

drop_cols = ['code','con_code_x','in_date_x','out_date_x','con_code_y',
         'in_date_y','out_date_y','open','high','low','pre_close','pct_chg',
        'vol_y','amount','change']

for count, i in enumerate(dates):

    #数据
    df_sh = pro.hk_hold(trade_date = i,exchange = 'SH')
    df_sz = pro.hk_hold(trade_date = i,exchange = 'SZ')

    # 股票量价数据
    data = pd.concat([df_sh,df_sz])
    df_price = pro.daily(trade_date=i)

    data = data.merge(df_price, how='inner', on=['ts_code','trade_date'])
    

    # 数据合并
    data = data.merge(df_first_detail, how='inner', left_on ="ts_code",right_on='con_code')
    data = data.merge(df_second_detail, how='inner', left_on ="ts_code",right_on='con_code')

    # 数据清洗
    data = data.drop(drop_cols,axis=1)

    data_all = pd.concat([data_all,data])   
    
    if count%10 == 0:
        time.sleep(60)

print(f"{data_all['trade_date'].iloc[0]} 至 {data_all['trade_date'].iloc[-1]}的数据已经提取完毕")

data = data_all

20211119 至 20211112的数据已经提取完毕
CPU times: user 723 ms, sys: 91.7 ms, total: 814 ms
Wall time: 1min 31s


In [9]:
%%time
data.columns = ['date','code','name','north_holding','ratio','exchange','close','sw_first','sw_second']

shares = pd.unique(data['code'])

data_net_hd = pd.DataFrame()
for i in range(len(shares)):
    share_table = data[data['code'] == shares[i]]
    share_table = share_table.sort_values(by='date',ascending=True)
    share_table['net_hd'] = share_table['north_holding'].diff()
    data_net_hd = pd.concat([data_net_hd,share_table])


data_net_hd['net_cf'] = data_net_hd['net_hd']* data_net_hd['close']

# 去除NaN值
data = data_net_hd.dropna()

CPU times: user 2.95 s, sys: 25.5 ms, total: 2.98 s
Wall time: 2.98 s


In [10]:
%%time
# 创建一个全日期的空序列
share_pivot = pd.DataFrame(pd.unique(data['date']), columns={'date'})

shares = pd.unique(data['code'])

for share in shares:
    share_net_cf = data[data['code'] == share][['date','net_cf']].sort_values(by = 'date',ascending=True)

    share_net_cf = share_net_cf.rename(columns={'net_cf': share})

    share_pivot = share_pivot.merge(share_net_cf,how = 'outer',on = 'date')
    
# recording
    if share_pivot.shape[1] % 500 == 0:
        print(share_pivot.shape[1])

500
1000
1500
CPU times: user 1min 2s, sys: 352 ms, total: 1min 2s
Wall time: 1min 2s


In [11]:
share_pivot = share_pivot.fillna(0)

# share_pivot = pivot_data(data,'code','date','net_cf')

share_pivot['date'] = pd.to_datetime(share_pivot['date'],format= "%Y%m%d")

share_pivot =  share_pivot.set_index('date',drop = True)

In [12]:
share_pivot_w = share_pivot.resample("W-FRI").sum()

In [13]:
sw_first = pivot_data(data,'sw_first','date','net_cf')

sw_first['date'] = pd.to_datetime(sw_first['date'],format='%Y%m%d')
sw_first = sw_first.set_index('date',drop=True)

In [14]:
sw_first_w = sw_first.resample("W-FRI").sum()

## 筛选行业

In [15]:
industry_code_w = find_top(top_industry,sw_first_w)

## 筛选股票

In [16]:
share_code = pd.DataFrame()

for each_day in range(industry_code_w.shape[1]):

    share_code_w = pd.DataFrame()

    for each_industry in industry_code_w.iloc[:,each_day]:

        #     获取分类的成份股
        industry_share = pd.unique(data[data['sw_first'] == each_industry]['code'])

        #     每个行业按照现金流净流入选出top n个股票
        share = share_pivot_w.loc[industry_code_w.columns[each_day],industry_share].sort_values(ascending = False).index[:top_share]

        share = pd.DataFrame(share,columns=[industry_code_w.columns[each_day]])

        
        share_code_w = pd.concat([share_code_w,share],axis = 0)
    share_code = pd.concat([share_code,share_code_w],axis = 1)
    if each_day % 20 == 0:
        print(each_day)

0


In [17]:
share_code = share_code.reset_index(drop=False)

In [18]:
result = pd.DataFrame()

# 记录rand和股票code
result['rank'] = share_code.iloc[:,0]
result['code'] = share_code.iloc[:,1]

share_name = data[['code','name']]

# 合并申万行业、股票名称
result = result.merge(data[['code','name']].drop_duplicates(),how='left',left_on='code',right_on='code')
result = result.merge(df_first_detail,how='left',left_on='code',right_on='con_code')
result = result.merge(df_second_detail,how='left',left_on='code',right_on='con_code')

share_name = data[['code','name']]

result = result.iloc[:,[0,1,2,3,7]]

result.columns = ['rank','code','name','sw_first_code','sw_second_code']

# 补充申万行业名称
sw_classify_first = pro.index_classify(level='L1', src='SW2021')
sw_classify_second = pro.index_classify(level='L2', src='SW2021')

result = result.merge(sw_classify_first[['index_code','industry_name']],how='left',left_on='sw_first_code',right_on='index_code')
result = result.merge(sw_classify_second[['index_code','industry_name']],how='left',left_on='sw_second_code',right_on='index_code')
result = result.drop(['sw_first_code','sw_second_code'],axis = 1)

result = result.rename(columns={'index_code_x': 'sw_first_code', 'industry_name_x': 'sw_first','index_code_y':'sw_second_code','industry_name_y':'sw_second'})

In [19]:
# result.to_csv('资金流向股票.csv',encoding='utf_8_sig')
# result

# 输出股票代码

In [22]:
last_week_data = pd.read_csv('输出\股票代码.csv',index_col=0)

In [23]:
selected_share = [0,5,19, 20, 18, 22, 17, 24, 21, 12]
result['date'] = end_date
result.iloc[selected_share,:].to_csv('输出\股票代码.csv',encoding='utf_8_sig')
result.iloc[selected_share,:]

,rank,code,name,sw_first_code,sw_first,sw_second_code,sw_second,date
0,0,002241.SZ,歌爾股份,801080.SI,电子,801085.SI,消费电子,20211119
5,0,300750.SZ,寧德時代,801730.SI,电力设备,801733.SI,其他电源设备Ⅱ,20211119
19,4,002812.SZ,恩捷股份,801030.SI,基础化工,801034.SI,化学制品,20211119
20,0,002352.SZ,順豐控股,801170.SI,交通运输,801178.SI,物流,20211119
18,3,600141.SH,興發集團,801030.SI,基础化工,801034.SI,化学制品,20211119
22,2,600233.SH,圓通速遞,801170.SI,交通运输,801178.SI,物流,20211119
17,2,002326.SZ,永太科技,801030.SI,基础化工,801034.SI,化学制品,20211119
24,4,600704.SH,物產中大,801170.SI,交通运输,801178.SI,物流,20211119
21,1,002010.SZ,傳化智聯,801170.SI,交通运输,801178.SI,物流,20211119
12,2,600436.SH,PIEN TZE HUAN-A,801150.SI,医药生物,801155.SI,中药Ⅱ,20211119


# 更新上周股票收益率

In [25]:
last_week_data

In [ ]:
last_week_return = pd.DataFrame()
for code in last_week_data['code']:

    df = pro.weekly(ts_code=code, trade_date = end_date)

    last_week_return = pd.concat([last_week_return,df])

In [ ]:
last_week_return.index = last_week_data.index

In [ ]:
last_week_return[['ts_code','trade_date','pct_chg']].to_csv('输出\组合收益率.csv')

# 更新HS300指数收益率

In [ ]:
index_return = pro.index_weekly(ts_code='000300.SH', trade_date = end_date)

In [ ]:
index_return['trade_date'] =  pd.to_datetime(index_return['trade_date'])

In [ ]:
index_return[['trade_date','pct_chg']].to_csv('输出\HS300收益率.csv')